In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import trapezoid
from joblib import Parallel, delayed
import time

#Here we will solve for the size-frequencies of our coral population

#First define the size of the landscape
m = 10 #We will live in a 10x10 m sandbox
Am = (10*100) ** 2 #Total area in cm 

#Retreive rData
%store -r rData
dr = rData[1]-rData[0]
nr = len(rData)

#Time data
y = 450 #We will solve up to 450 years
dy = 1
yData = np.arange(0, y+dy, dy) #We will store size-frequency data each yr
T = 365 * y #We will convert to days 
dt = 5 #We will increment time over 5 days
tData = np.arange(0, T+dt, dt)

#Growth function
def growth(a):
    gamma = 2/365 #Base growth rate for coral without crowding
    nuGrowth = 2 #Exponent of occupied ratio 
    return gamma*(1-(a ** nuGrowth))

#Mortality function
def mortality(r):
    Lmin = 2.5 * 365 #Min life-expectancy is 2.5 years
    Lmax = 20 * 365 #Max life-expectancy is 20 years
    thetaMort = 20 #Radius value where life expectancy transitions from 
                    #Lmin to Lmax
    betaMort = 2 #Transition parameter of life-expectancy Hill-equation
    LE = Lmin + (Lmax-Lmin)*((r ** betaMort)/((thetaMort ** betaMort)+(r**betaMort)))
    return 1/LE

mort = mortality(rData)


#CAFI equations
#Retreive effective CAFI densities
%store -r X 
def CAFIHill(phi,X,a):
    areaFrac = np.around(np.linspace(0,1,1001),decimals = 3 )
    aRound = np.around(a, decimals = 3)
    XEffective = X[areaFrac == aRound]
    
    rTheta = 20
    rThetaIndx = np.where(rData == rTheta)[0]
    aTheta = 0.5
    aThetaIndx = np.where(areaFrac == aTheta)[0]
    thetaCAFIHill = X[rThetaIndx, aThetaIndx] #Corresponding to CAFI 
                                                #density of half filled
                                                #environment at 20 cm
    
    betaCAFIHill = 2 #transition parameter of CAFI density Hill-equation
    return 1 + phi*( (XEffective ** betaCAFIHill) / ( (thetaCAFIHill ** betaCAFIHill) + (XEffective ** betaCAFIHill) ) )

phiValues = np.array( [-0.5 , 0, 0.5])

#dXdr = np.gradient(X, axis = 1)

#Immigration
def Imm(a):
    newCorals = 42
    days = 28
    Gamma = newCorals/days
    nuImm = 2
    return Gamma*(1-(a**nuImm))


#Initial population information, adjust where necessary, but we will
#start with empty landscapes
u0 = np.zeros(nr)

def sizeFreq_forPhi(phi):
    print('Solving for phi =', phi)
    u = u0
    for t in tData:
       
        if t/365 == round(t/365):
            year = round(t/365)
            print(f'\rSolving for year = {year:.0f}', end='')
            if t == 0:
                uSave = u
            else:
                uSave = np.vstack([uSave, u])

        A = trapezoid(np.pi * (rData ** 2) * u, rData)
        a = A/Am
        hill = CAFIHill(phi, X, a)
        hill = hill[0]
        growthT = growth(a) * hill
        mortT = mort * hill

        dHilldr = np.gradient(hill)
        dgdr = growth(a) * dHilldr

        u0Prev = u[0]
        if a >= 1:
            u[0] = 0
        else:
            u[0] = (dt*Imm(a))/dr + u[0] - dt * (growthT[0]/dr + mortT[0] +dgdr[0] )* u[0]

        if sum(growthT > dr/dt) > 0 :
            raise ValueError("CFL condition broken")
        else:
            Amat = (1-dt*growthT[1:]/dr-dt*mortT[1:]-dt*dgdr[1:])*np.eye(nr-1) + (dt*growthT[1:]/dr)*np.eye(nr-1, k = -1)
            uCol = u[1:].T
            uCol = Amat @ uCol
            u[1:] = uCol.T
            u[1] = u[1] + dt*growthT[0]*u0Prev/dr
    
    return uSave
    
print("Starting parallel computations...")
results = Parallel(n_jobs = -1, verbose = 3)(delayed(sizeFreq_forPhi)(phi=P) for P in phiValues)

uSizeFreq = np.stack(results)
print("All computations are complete.")           


Starting parallel computations...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
